In [1]:
import random

In [2]:
def build_dictionary(dictionary_file_location):
    """
    Builds a dictionary mapping word lengths to lists of words with that length.
    """
    word_dict = {}
    with open(dictionary_file_location, "r") as text_file:
        for line in text_file:
            word = line.strip()
            word_length = len(word)
            if word_length not in word_dict:
                word_dict[word_length] = []
            word_dict[word_length].append(word)
    return word_dict


def choose_secret_word(word_dict):
    """
    Selects a random word from the dictionary based on frequency distribution.
    """
    total_words = sum(len(words) for words in word_dict.values())
    word_length = random.choices(list(word_dict.keys()), weights=[len(words) / total_words for words in word_dict.values()])[0]
    return random.choice(word_dict[word_length])


def calculate_letter_entropy(displayed_word, word_dict):
    """
    Calculates the entropy of each letter based on remaining possible words.
    """
    letter_entropies = {}
    for word_length, words in word_dict.items():
        if len(displayed_word) == word_length:
            for word in words:
                for letter in word:
                    if letter not in displayed_word and letter not in letter_entropies:
                        letter_entropies[letter] = 0
                    elif letter not in displayed_word:
                        letter_entropies[letter] += 1 / len(word)  # Average uncertainty reduction

    # Normalize entropies for comparison
    total_entropy = sum(entropies for entropies in letter_entropies.values())
    if total_entropy > 0:
        for letter, entropy in letter_entropies.items():
            letter_entropies[letter] /= total_entropy
    return letter_entropies


def make_best_guess(displayed_word, word_dict):
    """
    Finds the letter with the highest entropy based on remaining possibilities.
    """
    letter_entropies = calculate_letter_entropy(displayed_word, word_dict)
    return max(letter_entropies, key=letter_entropies.get)


def is_letter_present(guess, secret_word):
    """
    Checks if the guessed letter is present in the secret word.
    """
    return guess.lower() in secret_word.lower()


def update_word_display(guess, secret_word, displayed_word):
    """
    Updates the displayed word with correctly guessed letters.
    """
    updated_word = ""
    for idx, letter in enumerate(secret_word):
        if displayed_word[idx] == "_" and letter.lower() == guess.lower():
            updated_word += guess
        else:
            updated_word += displayed_word[idx]
    return updated_word


def update_possible_words(displayed_word, guess, is_correct, word_dict):
    """
    Updates the list of possible words based on the guess and its outcome.

    Args:
        displayed_word: The current display of the word with underscores.
        guess: The letter guessed by the user.
        is_correct: True if the guess was correct, False otherwise.
        word_dict: Dictionary of word lengths and corresponding word lists.

    Returns:
        An updated dictionary with word lengths as keys and filtered word lists as values.
    """

    # Create a copy of the word dictionary to avoid modifying the original
    updated_word_dict = {word_length: words.copy() for word_length, words in word_dict.items()}

    # Filter word lists based on correct/incorrect guess
    if is_correct:
        # Keep words containing the guessed letter at the correct positions
        for word_length, words in updated_word_dict.items():
            if len(displayed_word) == word_length:
                updated_word_dict[word_length] = [word for word in words if all(displayed_word[idx] == word[idx] or displayed_word[idx] == "_" for idx in range(word_length))]
    else:
        # Keep words NOT containing the guessed letter
        for word_length, words in updated_word_dict.items():
            if len(displayed_word) == word_length:
                updated_word_dict[word_length] = [word for word in words if guess not in word]

    return updated_word_dict

def start_game(word_dict, num_games):
    """
    Runs the Hangman game multiple times and calculates accuracy.
    """

    total_success = 0
    for _ in range(num_games):
        secret_word = choose_secret_word(word_dict)
        displayed_word = "_" * len(secret_word)
        max_guesses = 6
        word_dict = {word_length: words.copy() for word_length, words in word_dict.items()}  # Create a copy for each game

        while True:
            if displayed_word == secret_word:
                total_success += 1
                print(f"You won! The word was {secret_word}.")
                break
            if max_guesses == 0:
                print(f"You lost! The word was {secret_word}.")
                break

            guess = make_best_guess(displayed_word, word_dict)

            if is_letter_present(guess, secret_word):
                print(f"Good guess! '{guess}' is in the word.")
                displayed_word = update_word_display(guess, secret_word, displayed_word)
            else:
                print(f"Oops! '{guess}' is not in the word.")
                max_guesses -= 1

            word_dict = update_possible_words(displayed_word, guess, is_letter_present(guess, secret_word), word_dict)

            print(f"Current word: {displayed_word}")
            print(f"You have {max_guesses} guesses left.")

    accuracy = (total_success / num_games) * 100
    print(f"Accuracy: {accuracy:.2f}%")

    return accuracy



In [3]:
dictionary_file_location = "RandomWords.txt"

In [4]:
word_dict = build_dictionary(dictionary_file_location)

In [6]:
num_games = 1000  # Play the game 100 times

start_game(word_dict, num_games)

Streaming output truncated to the last 5000 lines.
Current word: c_c__tri_et___enetrinitr__ine
You have 6 guesses left.
Good guess! 'y' is in the word.
Current word: cyc__tri_et_y_enetrinitr__ine
You have 6 guesses left.
Good guess! 'l' is in the word.
Current word: cycl_tri_et_ylenetrinitr__ine
You have 6 guesses left.
Good guess! 'm' is in the word.
Current word: cycl_trimet_ylenetrinitr_mine
You have 6 guesses left.
Good guess! 'o' is in the word.
Current word: cyclotrimet_ylenetrinitr_mine
You have 6 guesses left.
Good guess! 'h' is in the word.
Current word: cyclotrimethylenetrinitr_mine
You have 6 guesses left.
Good guess! 'a' is in the word.
Current word: cyclotrimethylenetrinitramine
You have 6 guesses left.
You won! The word was cyclotrimethylenetrinitramine.
Good guess! 'r' is in the word.
Current word: r_____r
You have 6 guesses left.
Good guess! 'e' is in the word.
Current word: re___er
You have 6 guesses left.
Good guess! 's' is in the word.
Current word: res__er
You have 

99.4